## Loading the dataset

In [35]:
from loguru import logger
import sys

logger.remove()
logger.add(sys.stdout, level="INFO")  # or "DEBUG" for even more


10

In [73]:

import pandas as pd
from sklearn.model_selection import train_test_split


csv_path = "letter_a_b.csv" 
TARGET_COLUMN = 'letter'
df = pd.read_csv(csv_path)


df = df.dropna()

print(df.head())
print(df.info())


  letter  xbox  ybox  width  height  onpix  xbar  ybar  x2bar  y2bar  xybar  \
0      B     4     2      5       4      4     8     7      6      6      7   
1      A     1     1      3       2      1     8     2      2      2      8   
2      B     5     9      7       7     10     9     8      4      4      6   
3      A     3     7      5       5      3    12     2      3      2     10   
4      A     3     8      5       6      3     9     2      2      3      8   

   x2ybar  xy2bar  xedge  xedgey  yedge  yedgex  
0       6       6      2       8      7      10  
1       2       8      1       6      2       7  
2       8       6      6      11      8       7  
3       2       9      2       6      3       8  
4       2       8      2       6      3       7  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1555 entries, 0 to 1554
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   letter  1555 non-null   object
 1   xbox 

### Initialise PATE-GAN

In [74]:
from synthcity.plugins import Plugins

pategan = Plugins().get(
    "pategan",
    # --- Differential Privacy ---
    epsilon=2.0,                # less strict for better utility
    delta=1e-5,
    lamda=0.01,
    n_teachers=1,
    teacher_template="xgboost",
    clipping_value=1,

    # --- Generator Config ---
    generator_n_layers_hidden=5,
    generator_n_units_hidden=256,
    generator_nonlin="leaky_relu",
    generator_dropout=0.2,
    generator_n_iter=25,  # more updates per loop

    # --- Discriminator Config ---
    discriminator_n_layers_hidden=5,
    discriminator_n_units_hidden=256,
    discriminator_nonlin="leaky_relu",
    discriminator_dropout=0.2,
    discriminator_n_iter=25,

    # --- Training ---
    n_iter=100,
    lr=1e-4,
    
    weight_decay=1e-5,
    batch_size=1024,
    random_state=42,

    # --- Encoding ---
    encoder_max_clusters=10,
)


2025-05-19 01:16:45.484 | CRITICAL | synthcity.logger:log_and_print:65 - load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'
2025-05-19 01:16:45.485 | CRITICAL | synthcity.logger:log_and_print:65 - load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'
2025-05-19 01:16:45.486 | CRITICAL | synthcity.logger:log_and_print:65 - module plugin_great load failed
2025-05-19 01:16:45.487 | CRITICAL | synthcity.logger:log_and_print:65 - module disabled: /home/seyam-omar/jupyenv/lib/python3.12/site-packages/synthcity/plugins/generic/plugin_goggle.py


### Fitting pate-gan

In [75]:
from synthcity.plugins.core.dataloader import GenericDataLoader
from synthcity.plugins.core.schema import Schema

# Define the target column
target_col = TARGET_COLUMN

# Identify all feature columns
feature_cols = [col for col in df.columns if col != target_col]

# Create a domain dictionary marking all features as numerical
domain = {col: "numerical" for col in feature_cols}

# Initialize the schema with the domain
schema = Schema(domain=domain)

# Create the data loader with the specified schema
dataloader = GenericDataLoader(df, target_column=target_col, schema=schema)


# Fit the generator
pategan.fit(dataloader)

print("Training complete. Fitted status:", pategan.fitted)


2025-05-19 01:16:48.281 | INFO     | synthcity.logger:log_and_print:65 - [pategan] using delta = 1.6308139207105265e-05
2025-05-19 01:16:48.286 | INFO     | synthcity.logger:log_and_print:65 - Encoding letter 5767584641373809913
2025-05-19 01:16:48.290 | INFO     | synthcity.logger:log_and_print:65 - Encoding xbox 12695041101220943635
2025-05-19 01:16:48.979 | INFO     | synthcity.logger:log_and_print:65 - Encoding ybox 13921920367518933482
2025-05-19 01:16:49.676 | INFO     | synthcity.logger:log_and_print:65 - Encoding width 6687162974100422244
2025-05-19 01:16:49.955 | INFO     | synthcity.logger:log_and_print:65 - Encoding height 7386505415593150607
2025-05-19 01:16:49.962 | INFO     | synthcity.logger:log_and_print:65 - Encoding onpix 9598955325056794481
2025-05-19 01:16:50.487 | INFO     | synthcity.logger:log_and_print:65 - Encoding xbar 8496188470491360011
2025-05-19 01:16:50.496 | INFO     | synthcity.logger:log_and_print:65 - Encoding ybar 5798536606865221659
2025-05-19 01:16

100%|███████████████████████████████████████████| 25/25 [03:10<00:00,  7.61s/it]

2025-05-19 01:20:03.539 | INFO     | synthcity.logger:log_and_print:65 - [pategan it 1] epsilon_hat = 171.84304623692026. self.epsilon = 2.0
Training complete. Fitted status: True


## Generate Synthetic data

In [76]:
import pandas as pd

TARGET_COLUMN='letter'
# --- Configuration ---
target_col = TARGET_COLUMN        # your real target column
total_samples = int(len(df) * 0.5)
unique_classes = df[target_col].unique()
num_classes = len(unique_classes)
desired_per_class = total_samples // num_classes

# Accumulate per‐class chunks here
accumulated = {cls: [] for cls in unique_classes}

# You can tweak this batch size up or down
batch_size = desired_per_class * num_classes

# Keep generating until every class has enough rows
while True:
    # Check if we’re done
    if all(
        sum(len(chunk) for chunk in accumulated[cls]) >= desired_per_class
        for cls in unique_classes
    ):
        break

    # Generate a fresh batch (unconditional)
    batch_loader = pategan.generate(count=batch_size)
    batch_df = batch_loader.dataframe()

    # For each class, grab as many as still needed (if any)
    for cls in unique_classes:
        current_count = sum(len(chunk) for chunk in accumulated[cls])
        need = desired_per_class - current_count
        if need > 0:
            cls_rows = batch_df[batch_df[target_col] == cls]
            if not cls_rows.empty:
                # take up to `need` rows
                accumulated[cls].append(cls_rows.iloc[:need])

# Stitch together exactly `desired_per_class` rows of each class
parts = []
for cls in unique_classes:
    cls_df = pd.concat(accumulated[cls], ignore_index=True)
    parts.append(cls_df)

synthetic_df = pd.concat(parts, ignore_index=True)
synthetic_df = synthetic_df.sample(frac=1, random_state=42)  # shuffle

# Verify
print("Total synthetic rows:", len(synthetic_df))
print("Per-class counts:")
print(synthetic_df[target_col].value_counts())


Total synthetic rows: 776
Per-class counts:
letter
A    388
B    388
Name: count, dtype: int64


### TSTR

In [77]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.pipeline import Pipeline

# --- Configuration ---

TEST_SIZE = 0.3
RANDOM_STATE = 42

# Assume synthetic_df is already loaded in the environment
# Validate target column
if TARGET_COLUMN not in synthetic_df.columns:
    raise ValueError(f"Target column '{TARGET_COLUMN}' not found.")

# Separate features and target
X_original = synthetic_df.drop(TARGET_COLUMN, axis=1)
y_original = synthetic_df[TARGET_COLUMN]

# Encode target
le = LabelEncoder()
y = le.fit_transform(y_original)
print(f"Class mapping: {dict(zip(le.classes_, le.transform(le.classes_)))}")

# Encode features
non_numeric_cols = X_original.select_dtypes(exclude=['number', 'bool']).columns
if len(non_numeric_cols) > 0:
    X = pd.get_dummies(X_original, columns=non_numeric_cols, drop_first=True, dummy_na=False)
else:
    X = X_original

# Train-test split
try:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
    )
except ValueError:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
    )

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define models
models = {
    "Logistic Regression": LogisticRegression(random_state=RANDOM_STATE, max_iter=1000),
    "Random Forest": RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=100),
    "XGBoost": xgb.XGBClassifier(random_state=RANDOM_STATE, use_label_encoder=False, eval_metric='logloss'),
    "MLP": MLPClassifier(random_state=RANDOM_STATE, max_iter=500, early_stopping=True, hidden_layer_sizes=(64, 32))
}

# 3-Fold Cross-Validation
print("\n--- 3-Fold Cross-Validation Results (All Folds) ---")
pipelines = {
    name: Pipeline([('scaler', StandardScaler()), ('clf', model)])
    for name, model in models.items()
}
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision_weighted',
    'recall': 'recall_weighted',
    'f1': 'f1_weighted',
}
cv_results = {}
for name, pipe in pipelines.items():
    scores = cross_validate(pipe, X, y, cv=3, scoring=scoring, return_train_score=False)
    
    print(f"\n{name} - Per Fold Scores:")
    for i in range(3):  # 3 folds
        print(f"  Fold {i+1}:")
        for metric in scoring:
            score_value = scores[f'test_{metric}'][i]
            print(f"    {metric.capitalize()}: {score_value:.4f}")
    
    avg_scores = {metric: np.mean(scores[f'test_{metric}']) for metric in scoring}
    cv_results[name] = avg_scores

cv_df = pd.DataFrame(cv_results).T
print("\nCross-Validation Summary (Averaged Across Folds):")
print(cv_df)



Class mapping: {'A': 0, 'B': 1}

--- 3-Fold Cross-Validation Results (All Folds) ---

Logistic Regression - Per Fold Scores:
  Fold 1:
    Accuracy: 0.5174
    Precision: 0.5176
    Recall: 0.5174
    F1: 0.5169
  Fold 2:
    Accuracy: 0.4981
    Precision: 0.4979
    Recall: 0.4981
    F1: 0.4969
  Fold 3:
    Accuracy: 0.4574
    Precision: 0.4569
    Recall: 0.4574
    F1: 0.4558

Random Forest - Per Fold Scores:
  Fold 1:
    Accuracy: 0.4865
    Precision: 0.4865
    Recall: 0.4865
    F1: 0.4865
  Fold 2:
    Accuracy: 0.4672
    Precision: 0.4672
    Recall: 0.4672
    F1: 0.4667
  Fold 3:
    Accuracy: 0.4612
    Precision: 0.4606
    Recall: 0.4612
    F1: 0.4589

XGBoost - Per Fold Scores:
  Fold 1:
    Accuracy: 0.4865
    Precision: 0.4864
    Recall: 0.4865
    F1: 0.4863
  Fold 2:
    Accuracy: 0.5290
    Precision: 0.5290
    Recall: 0.5290
    F1: 0.5289
  Fold 3:
    Accuracy: 0.4690
    Precision: 0.4689
    Recall: 0.4690
    F1: 0.4686

MLP - Per Fold Scores:
  Fold

## JSD and WD

In [78]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import jensenshannon

# --- Compute JSD and WD for each categorical column ---
jsd_results = {}
wd_results = {}

for col in df.columns:
    # Real and synthetic PMFs
    real_freq  = df[col].value_counts(normalize=True)
    synth_freq = synthetic_df[col].value_counts(normalize=True)

    # Union of categories
    cats = sorted(set(real_freq.index).union(synth_freq.index))

    # Build probability vectors (add epsilon to avoid zeros)
    eps = 1e-12
    P = np.array([real_freq.get(c, 0.0) for c in cats]) + eps
    Q = np.array([synth_freq.get(c, 0.0) for c in cats]) + eps
    P /= P.sum()
    Q /= Q.sum()

    # Jensen–Shannon Divergence
    jsd_results[col] = jensenshannon(P, Q) ** 2

    # Categorical WD with 0–1 cost is half the L1 distance
    wd_results[col] = 0.5 * np.abs(P - Q).sum()

# --- Compute and print averages ---
avg_jsd = np.mean(list(jsd_results.values()))
avg_wd  = np.mean(list(wd_results.values()))

print(f"Average JSD across all columns: {avg_jsd:.6f}")
print(f"Average WD  across all columns: {avg_wd:.6f}")


Average JSD across all columns: 0.018843
Average WD  across all columns: 0.114645
